# Input and Output

AMS supprots multiple input and output formats.

Still, first import the `ams` library and configure the logger level.

In [1]:
import os

import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2024-01-20 07:07:12
ams:0.8.0.post21+gdce75d5


In [3]:
ams.config_logger(stream_level=20)

## Input

### AMS Execel

In [4]:
sp_xlsx = ams.load(ams.get_case('ieee14/ieee14_uced.xlsx'),
                  setup=True,
                  no_output=True,)

sp_xlsx.summary()

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14_uced.xlsx"...
Input file parsed in 0.1516 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0034 seconds.
-> Systen size:
Base: 100 MVA; Frequency: 60 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.24 p.u.; Reactive load: 0.95 p.u.
-> Data check results:
ACED: ACOPF
DCED: DCOPF, ED, RTED
DCUC: UC
DED: DOPF
PF: DCPF, PFlow, CPF


### AMS JSON

In [5]:
sp_json = ams.load(ams.get_case('ieee14/ieee14.json'),
                  setup=True,
                  no_output=True,)

sp_json.summary()

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14.json"...
Input file parsed in 0.0035 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0052 seconds.
-> Systen size:
Base: 100 MVA; Frequency: 60 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.24 p.u.; Reactive load: 0.95 p.u.
-> Data check results:
PF: DCPF, PFlow, CPF


### MATPOWER

In [6]:
sp_mp = ams.load(ams.get_case('matpower/case14.m'),
                 setup=True,
                 no_output=True,)

sp_mp.summary()

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m"...
Input file parsed in 0.0218 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0075 seconds.
-> Systen size:
Base: 100.0 MVA; Frequency: 60 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.59 p.u.; Reactive load: 0.74 p.u.
-> Data check results:
ACED: ACOPF
DCED: DCOPF
DED: DOPF
PF: DCPF, PFlow, CPF


Note that AMS also supports PYPOWER format py-file.

### PSS/E RAW

AMS also supports PSS/E RAW format for power flow analysis.

In [7]:
sp_raw = ams.load(ams.get_case('ieee14/ieee14.raw'),
                  setup=True,
                  no_output=True,)

sp_raw.summary()

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14.raw"...
Input file parsed in 0.0123 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0031 seconds.
-> Systen size:
Base: 100.0 MVA; Frequency: 60.0 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.24 p.u.; Reactive load: 0.95 p.u.
-> Data check results:
PF: DCPF, PFlow, CPF


## Output

Vice versa, AMS supports multiple output formats.

In [8]:
ams.io.xlsx.write(system=sp_xlsx,
                  outfile='out.xlsx',)

xlsx file written to "out.xlsx"


True

In [9]:
os.remove('out.xlsx')

Similarly, JSON output formats can be achieved by using `ams.io.json.write`.